In [13]:
# Import statements
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import datetime

# Changes directory to location of ICM output files
os.chdir('C:\\Users\\03614\\PythonStuff\\ICM Results\\Free Outfall')

# Stores current working directory and filenames as lists
cwd = os.path.abspath('') 
files = os.listdir(cwd) 

# Determining number of csv (model output files) in directory and storing as i
i = len([name for name in os.listdir('.')
     if os.path.isfile(name)])

# Creates series object of csv filenames, to be applied as column names for dataframe
filenames = pd.Series(files)
linknums = filenames.str.slice(start=0,stop=8, step=1)
linknums = linknums.replace('-','', regex=True)

# Looping through cwd and creating lists for every file that ends in '.csv'
dfs = [pd.read_csv(f,index_col=[0], parse_dates=[0])
        for f in os.listdir(os.getcwd()) 
           if f.endswith('.csv')]

# Concatinating lists into a single dataframe, cleaning data, setting datatype
finaldf = pd.concat(dfs, axis=1, join='inner').sort_index()
finaldf.drop(columns=['Seconds'], inplace=True)
finaldf.drop(finaldf.tail(1).index, inplace=True)
#finaldf.index = finaldf.index.str.slice(start=11,stop=16,step=1)

finaldf = finaldf.apply(pd.to_numeric)

# Setting column names based on linknums series
finaldf.columns = linknums

# Create timeseries based on first index entry in dataframe, and reassigning df index
dti = pd.date_range(finaldf.index[0], 
                    periods = 865,
                    freq = '5T' )
finaldf.index = dti


# Loop through all the dfs, drop 'Time', 'Seconds', apply the index
for 

dfs[1]

,Seconds,WWTPLS1.1
Time,,
[Hr:Min:s],[s],ds_flow [MGD]
01/01/2001 00:00:00,0,24.18239
01/01/2001 00:05:00,300,24.00445
01/01/2001 00:10:00,600,22.10998
01/01/2001 00:15:00,900,21.55783
...,...,...
01/03/2001 23:40:00,258000,34.25177
01/03/2001 23:45:00,258300,37.86174
01/03/2001 23:50:00,258600,37.20886


In [4]:
# Change working directory so that output excel file can be written outside data repository 
os.chdir('C:\\Users\\03614\\PythonStuff\\ICM Results')

# Create a Pandas Excel writer using XlsxWriter as the engine.
excel_file = 'ICM_Results_Test.xlsx'
sheet_name = 'Sheet1'

writer = pd.ExcelWriter(excel_file, 
                        engine ='xlsxwriter', 
                        datetime_format = 'mm/dd/yyyy hh:mm:ss', 
                        date_format =  'mm/dd/yyyy'
        )

finaldf.to_excel(writer, sheet_name=sheet_name)

# Access the XlsxWriter workbook and worksheet objects from the dataframe.
workbook = writer.book
worksheet = writer.sheets[sheet_name]

# Adjust the width of the first column to make the date values clearer.
worksheet.set_column('A:A', 20)

# Create a chart object.
chart = workbook.add_chart({'type': 'scatter', 
                            'subtype': 'smooth'})

# Configure the series of the chart from the dataframe data.
max_row = len(finaldf) + 1
for i in range(len(linknums)):
    col = i + 1
    chart.add_series({
        'name':       ['Sheet1', 0, col],
        'categories': ['Sheet1', 1, 0, max_row, 0],
        'values':     ['Sheet1', 1, col, max_row, col],
        'line':       {'width': 1.50},
        'smooth':     True
    })

# Configure the chart axes and subobjects
chart.set_x_axis({'name': 'Time', 
                  'date_axis': True})

chart.set_y_axis({'name': 'Flow (MGD)',
                  'major_gridlines': {'visible': True}})



# Position the legend at the top of the chart.
chart.set_legend({'position': 'top'})

# Insert the chart into the worksheet.
worksheet.insert_chart('P2', chart)

# Close the Pandas Excel writer and output the Excel file.
writer.save()